In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pandas as pd

from IPython.display import display
pd.options.display.max_rows = 100
pd.options.display.max_colwidth = 150

## Источники данных

In [3]:
COVID_19_plus_Russia = 'grwlf/COVID-19_plus_Russia'
CSSEGISandData = 'CSSEGISandData/COVID-19'
EXTERNAL_DATA_DIRNAME = '../external'

In [31]:
#! git clone https://github.com/{COVID_19_plus_Russia}.git {EXTERNAL_DATA_DIRNAME}/{COVID_19_plus_Russia}

In [32]:
#! git clone https://github.com/{CSSEGISandData}.git {EXTERNAL_DATA_DIRNAME}/{CSSEGISandData}

## Чтение данных

### Информация по странам и регионам

In [4]:
countries = pd.read_csv(os.path.join('data', 'countries.csv'))
countries.drop(['iso_alpha2', 'iso_numeric', 'official_name', 'name'], axis='columns', inplace=True)

regions = pd.read_csv(os.path.join('data', 'russia_regions.csv'))
regname2iso3 = dict(zip(regions.name_with_type, regions.iso_code))
regions = regions[['iso_code', 'csse_province_state', 'population', 'population_urban']]
regions['population_urban'] = regions['population_urban'] / regions['population']
regions.rename(
    columns={'iso_code': 'iso_alpha3', 'csse_province_state': 'ccse_name', 'population_urban': 'urban_pop_rate'},
    inplace=True
)


countries = pd.concat([countries, regions], sort=True)

country2iso3 = dict(zip(countries['ccse_name'], countries['iso_alpha3']))
iso32country = dict(zip(countries['iso_alpha3'], countries['ccse_name']))

In [5]:
display(countries.head())
display(countries.tail())

,ccse_name,density,fertility_rate,iso_alpha3,land_area,median_age,migrants,population,urban_pop_rate,world_share
0,Afghanistan,60.0,4.6,AFG,652860.0,18.0,-62920.0,38928346,0.25,0.0050
1,Albania,105.0,1.6,ALB,27400.0,36.0,-14000.0,2877797,0.63,0.0004
2,Algeria,18.0,3.1,DZA,2381740.0,29.0,-10000.0,43851044,0.73,0.0056
3,Andorra,164.0,NaN,AND,470.0,NaN,NaN,77265,0.88,0.0000
4,Angola,26.0,5.6,AGO,1246700.0,17.0,6413.0,32866272,0.67,0.0042


,ccse_name,density,fertility_rate,iso_alpha3,land_area,median_age,migrants,population,urban_pop_rate,world_share
80,Cheliabinsk oblast,NaN,NaN,RU-CHE,NaN,NaN,NaN,3466960,0.826372,NaN
81,Republic of Chuvashia,NaN,NaN,RU-CU,NaN,NaN,NaN,1217820,0.634003,NaN
82,Chukotskiy autonomous oblast,NaN,NaN,RU-CHU,NaN,NaN,NaN,50726,0.715077,NaN
83,Yamalo-Nenetskiy AO,NaN,NaN,RU-YAN,NaN,NaN,NaN,544008,0.839333,NaN
84,Yaroslavl oblast,NaN,NaN,RU-YAR,NaN,NaN,NaN,1253189,0.815866,NaN


### Исторические данные по странам из CSSEGISandData/COVID-19

In [6]:
data = pd.read_csv(
    os.path.join(
        EXTERNAL_DATA_DIRNAME, CSSEGISandData,
        'csse_covid_19_data', 'csse_covid_19_time_series', 'time_series_covid19_confirmed_global.csv')
)
data = data.groupby('Country/Region').sum()

data_lat_long = data.reset_index().iloc[:, :3]
data_lat_long.columns = ['country', 'lat', 'long']

data = data.iloc[:,4:].stack().reset_index()
data.columns = ['country', 'date', 'confirmed']

data_d = pd.read_csv(os.path.join(
    EXTERNAL_DATA_DIRNAME, CSSEGISandData, 
    'csse_covid_19_data', 'csse_covid_19_time_series', 'time_series_covid19_deaths_global.csv'))
data_d = data_d.groupby('Country/Region').sum()
data_d = data_d.iloc[:,4:].stack().reset_index()
data_d.columns = ['country', 'date', 'deaths']

data = data_lat_long.merge(data).merge(data_d)
data['date'] = pd.to_datetime(data['date'])

### Данные за последний по странам из CSSEGISandData/COVID-19 (web-data ветка)

In [7]:
CURRENT_DATE = '2020-05-03'

In [8]:
df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv')

df_add = df[['Country_Region', 'Confirmed', 'Deaths']].copy()
df_add.columns = ['country', 'confirmed', 'deaths']

country2lat = dict(zip(data['country'], data['lat']))
country2long = dict(zip(data['country'], data['long']))
df_add['lat'] = df_add['country'].map(country2lat)
df_add['long'] = df_add['country'].map(country2long)

df_add['date'] = pd.to_datetime(CURRENT_DATE)
df_add[df_add['country'] == 'Russia']

,country,confirmed,deaths,lat,long,date
13,Russia,134687,1280,60.0,90.0,2020-05-03


### Полные данные по странам

In [9]:
data = pd.concat([data, df_add[data.columns]])
data = data.sort_values(['country', 'date']).reset_index(drop=True)
data['iso3'] = data['country'].map(country2iso3)

data[data.iso3 == 'RUS'].tail()

,country,lat,long,date,confirmed,deaths,iso3
14135,Russia,60.0,90.0,2020-04-29,99399,972,RUS
14136,Russia,60.0,90.0,2020-04-30,106498,1073,RUS
14137,Russia,60.0,90.0,2020-05-01,114431,1169,RUS
14138,Russia,60.0,90.0,2020-05-02,124054,1222,RUS
14139,Russia,60.0,90.0,2020-05-03,134687,1280,RUS


### Данные по регионам России из grwlf/COVID-19_plus_Russia

<h4>
    <p style="color: red">
        Files released after March 25 were updates with additional information on Russian regions.
    </p>
</h4>

In [10]:
data_Ru = pd.read_csv(os.path.join(EXTERNAL_DATA_DIRNAME, COVID_19_plus_Russia,
    'csse_covid_19_data', 'csse_covid_19_time_series', 'time_series_covid19_confirmed_RU.csv'
)).groupby("Province_State").sum()
data_Ru = data_Ru.iloc[:,15:].stack().reset_index()
data_Ru.columns = ['country', 'date', 'confirmed']

data_deaths_Ru = pd.read_csv(os.path.join(EXTERNAL_DATA_DIRNAME, COVID_19_plus_Russia,
    'csse_covid_19_data', 'csse_covid_19_time_series', 'time_series_covid19_deaths_RU.csv'
)).groupby('Province_State').sum()
data_deaths_Ru = data_deaths_Ru.iloc[:,15:].stack().reset_index()
data_deaths_Ru.columns = ['country', 'date', 'deaths']
data_Ru = data_Ru.merge(data_deaths_Ru, how = 'left', 
                        left_on=('country', 'date'), right_on=('country', 'date'))
data_Ru['recovered'] = 0

data_Ru.loc[data_Ru['country'] == 'Altay republic', 'country'] = 'Republic of Altay'

data_Ru['iso3'] = data_Ru['country'].map(country2iso3)
data_Ru['date'] = pd.to_datetime(data_Ru['date'])

## Посмотрим на имеющиеся данные ДО 25 марта

Есть только Москва и Питер!

In [11]:
data_Ru[(data_Ru['date'] > '2020-03-17') & (data_Ru['date'] <= '2020-03-24') 
        & (data_Ru['confirmed'] != 0)]

,country,date,confirmed,deaths,recovered,iso3
2899,Moscow,2020-03-18,87,0,0,RU-MOW
2900,Moscow,2020-03-19,99,0,0,RU-MOW
2901,Moscow,2020-03-20,132,0,0,RU-MOW
2902,Moscow,2020-03-21,138,0,0,RU-MOW
2903,Moscow,2020-03-22,191,0,0,RU-MOW
2904,Moscow,2020-03-23,262,0,0,RU-MOW
2905,Moscow,2020-03-24,290,0,0,RU-MOW
6129,Saint Petersburg,2020-03-18,11,0,0,RU-SPE
6130,Saint Petersburg,2020-03-19,12,0,0,RU-SPE
6131,Saint Petersburg,2020-03-20,16,0,0,RU-SPE


## 25 марта появляются данные по всем остальным регионам

Нам явно нужна правильная история, иначе в Московской области модель будет учиться прогнозировать рост 0 -> 41 человек за день. 

In [12]:
data_Ru[data_Ru['date'].isin(['2020-03-24', '2020-03-25']) 
        & (data_Ru['iso3'].isin(['RU-MOW', 'RU-MOS', 'RU-SVE', 'RU-SAM']))]

,country,date,confirmed,deaths,recovered,iso3
2905,Moscow,2020-03-24,290,0,0,RU-MOW
2906,Moscow,2020-03-25,410,0,0,RU-MOW
3000,Moscow oblast,2020-03-24,0,0,0,RU-MOS
3001,Moscow oblast,2020-03-25,41,0,0,RU-MOS
6325,Samara oblast,2020-03-24,0,0,0,RU-SAM
6326,Samara oblast,2020-03-25,10,0,0,RU-SAM
6800,Sverdlov oblast,2020-03-24,0,0,0,RU-SVE
6801,Sverdlov oblast,2020-03-25,13,0,0,RU-SVE


### Помогут данные из таблички Роспотребнадзора
https://docs.google.com/spreadsheets/d/1Re1TOpN32DY6_iFQNfY8D-bFqBw_gMQKZG7Mg7iRNqQ

In [13]:
data_rosp = pd.read_csv(os.path.join('data', 'ru_regions_rosp_data.tsv'), sep='\t', encoding='utf-8')
data_rosp = data_rosp.melt(id_vars=['name_with_type', 'iso_code'],  var_name='date',  value_name='confirmed')
data_rosp = data_rosp[data_rosp['date'] != 'Unnamed: 69'].reset_index(drop=True)
data_rosp = data_rosp[data_rosp['date'] != 'Unnamed: 70'].reset_index(drop=True)
data_rosp = data_rosp[data_rosp['date'] != 'Unnamed: 70'].reset_index(drop=True)
data_rosp = data_rosp[data_rosp['iso_code'] != 'KZ-BAY'].reset_index(drop=True)
data_rosp['date'] = pd.to_datetime(data_rosp['date'], format='%d.%m.%Y')
data_rosp = data_rosp[data_rosp['date'] <= CURRENT_DATE].sort_values('date').reset_index(drop=True)
data_rosp = data_rosp.fillna(0)
data_rosp['confirmed'] = data_rosp['confirmed'].astype(int)
data_rosp.columns = ['region_name', 'iso3', 'date', 'confirmed']

# Считаем кумулятивную сумму, т.к. исходные данные содержат прирост за каждый день
data_rosp['confirmed'] = data_rosp.groupby(['iso3'])['confirmed'].apply(lambda x: x.cumsum())
data_rosp = data_rosp[['iso3', 'date', 'confirmed']]

dates = list(data[data['country'] == 'Russia']['date'])
one_day = data_rosp[data_rosp['date'] == data_rosp.date.min()].reset_index(drop=True)
for dt in dates:
    if len(data_rosp[data_rosp['date'] == dt]) == 0:
        one_day['confirmed'] = 0
        one_day['date'] = dt
        data_rosp = pd.concat([data_rosp, one_day])
data_rosp = data_rosp.sort_values(['iso3', 'date']).reset_index(drop=True)

data_rosp['country'] = data_rosp['iso3'].map(iso32country)
data_rosp['deaths'] = 0
data_rosp['recovered'] = 0

### Сравним данные, имеющиеся на 25 марта в двух источниках

Видим, что по некоторым регионам данные за 25 марта НЕ сходятся

In [14]:
df25 = data_rosp[(data_rosp['date'] == '2020-03-25')][['date', 'iso3', 'country', 'confirmed']] \
    .merge(data_Ru[(data_Ru['date'] == '2020-03-25')][['date', 'iso3', 'country', 'confirmed']],
           on=['date', 'iso3', 'country'], suffixes=('_rosp', '_grwlf'))
df25[df25['confirmed_rosp'] != df25['confirmed_grwlf']]

,date,iso3,country,confirmed_rosp,confirmed_grwlf
41,2020-03-25,RU-MOS,Moscow oblast,37,41
42,2020-03-25,RU-MOW,Moscow,405,410
63,2020-03-25,RU-SPE,Saint Petersburg,20,21
72,2020-03-25,RU-TYU,Tumen oblast,2,3
82,2020-03-25,RU-ZAB,Zabaykalskiy kray,0,1


### Расхождения можно считать некритичными - поэтому подготовленный датасет строим так:
- до 25 марта берем данные из таблички Роспотребнадзора
- начиная с 25 марта берем данные у grwlf

In [15]:
data_Ru_all = pd.concat([
    data_rosp[(data_rosp['date'] < '2020-03-25')],
    data_Ru[(data_Ru['date'] >= '2020-03-25')],
], sort=True)

In [16]:
data_Ru_all[data_Ru_all.iso3 == 'RU-MOW'].tail()

,confirmed,country,date,deaths,iso3,recovered
2940,48426,Moscow,2020-04-28,479,RU-MOW,0
2941,53739,Moscow,2020-04-29,611,RU-MOW,0
2942,53739,Moscow,2020-04-30,611,RU-MOW,0
2943,57300,Moscow,2020-05-01,658,RU-MOW,0
2944,62658,Moscow,2020-05-02,695,RU-MOW,0


### Из-за ошибки в данных за 29 апреля - исправляем данные вручную

При этом колонка deaths за эту дату будет НЕкорректной

In [17]:
data_ru_regions_0429 = pd.read_csv(os.path.join('data', 'ru_regions_20200429_correct.csv'))
data_ru_regions_0429['date'] = pd.to_datetime(data_ru_regions_0429['date'])
data_ru_regions_0429['deaths'] = 0
data_ru_regions_0429['recovered'] = 0
data_Ru_all = data_Ru_all[pd.to_datetime(data_Ru_all['date']) != '2020-04-29'].reset_index(drop=True)
data_Ru_all = pd.concat([data_Ru_all, data_ru_regions_0429], sort=True)
data_Ru_all = data_Ru_all.sort_values(['country', 'date']).reset_index(drop=True)

In [18]:
data_Ru_all[data_Ru_all.iso3 == 'RU-MOW'].tail()

,confirmed,country,date,deaths,iso3,recovered
2995,48426,Moscow,2020-04-28,479,RU-MOW,0
2996,50646,Moscow,2020-04-29,0,RU-MOW,0
2997,53739,Moscow,2020-04-30,611,RU-MOW,0
2998,57300,Moscow,2020-05-01,658,RU-MOW,0
2999,62658,Moscow,2020-05-02,695,RU-MOW,0


### Данные по регионам России за сегодня

(сохраняем вручную с сайта Роспотребнадзора https://www.rospotrebnadzor.ru/about/info/news/news_details.php?ELEMENT_ID=14316)

формат:
```
1. Москва - 3570
2. Московская область - 709
3. Нижегородская область - 129
4. Санкт-Петербург - 114
5. Архангельская область - 68
...
```

In [19]:
import datetime

def process_today(today_filename, last_day, shift=0):
    df_add_ru = []
    with open(os.path.join('data', today_filename), encoding='utf-8') as fin:
        for line in fin:
            arr = line.replace('–', '-').strip().replace('г.', 'г').split('.')[-1].strip().split(' - ')
            arr[0] = arr[0].replace('область', 'обл')
            arr[0] = arr[0].replace('Республика', 'Респ')
            arr[0] = arr[0].replace('Москва', 'г Москва')
            arr[0] = arr[0].replace('Санкт-Петербург', 'г Санкт-Петербург')
            arr[0] = arr[0].replace('Чеченская Респ', 'Респ Чеченская')
            arr[0] = arr[0].replace('Респ Саха (Якутия)', 'Респ Саха /Якутия/')
            arr[0] = arr[0].replace('Чувашская Респ', 'Чувашская Республика - Чувашия')
            arr[0] = arr[0].replace('Респ Северная Осетия', 'Респ Северная Осетия - Алания')
            arr[0] = arr[0].replace('Кабардино-Балкарская Респ', 'Респ Кабардино-Балкарская')
            arr[0] = arr[0].replace('Карачаево-Черкесская Респ', 'Респ Карачаево-Черкесская')
            arr[0] = arr[0].replace('Ханты-Мансийский АО', 'Ханты-Мансийский Автономный округ - Югра')
            arr[0] = arr[0].replace('Кемеровская обл', 'Кемеровская область - Кузбасс')
            arr[0] = arr[0].replace('Удмуртская Респ', 'Респ Удмуртская')
            arr[0] = arr[0].replace('Еврейская АО', 'Еврейская Аобл')
            arr[0] = arr[0].replace('гСевастополь', 'г Севастополь')
            #arr[0] = arr[0].replace('Севастополь', 'г Севастополь')
            arr[0] = arr[0].replace('Ненецкий автономный округ', 'Ненецкий АО')
            if arr[0] not in regname2iso3:
                print(arr)
            last_day[regname2iso3[arr[0]]] += int(arr[1])
    df_add_ru = pd.DataFrame(list(last_day.items()), columns=['iso3', 'confirmed'])

    df_add_ru['date'] = pd.to_datetime(CURRENT_DATE)
    df_add_ru['date'] = df_add_ru['date'] + datetime.timedelta(days=-shift)    
    
    return df_add_ru

In [20]:
df_add_ru_list = []
last_day = dict(zip(data_Ru_all[data_Ru_all.date == max(data_Ru_all.date)].iso3, 
                    data_Ru_all[data_Ru_all.date == max(data_Ru_all.date)].confirmed))
df_add_ru_list.append(process_today('russia_today_0503.txt', last_day))

#### Добавляем последний день

In [21]:
data_Ru_final = pd.concat([data_Ru_all[['iso3', 'confirmed', 'date']]] + df_add_ru_list, sort=True)
data_Ru_final['country'] = data_Ru_final['iso3'].map(iso32country)
data_Ru_final = data_Ru_final.sort_values(['country', 'date']).reset_index(drop=True)

In [22]:
data_Ru_final[data_Ru_final.iso3 == 'RU-MOW'].tail()

,confirmed,date,iso3,country
3025,50646,2020-04-29,RU-MOW,Moscow
3026,53739,2020-04-30,RU-MOW,Moscow
3027,57300,2020-05-01,RU-MOW,Moscow
3028,62658,2020-05-02,RU-MOW,Moscow
3029,68606,2020-05-03,RU-MOW,Moscow


### Полные данные на текущий день: страны + регионы
Формат:
- date
- iso3
- country
- confirmed


In [23]:
data_full = pd.concat([data[['iso3', 'country', 'confirmed', 'date']], data_Ru_final], sort=True).reset_index(drop=True)
data_full = data_full[['date', 'iso3', 'country', 'confirmed']].sort_values(['country', 'date']).reset_index(drop=True)

In [24]:
data_full[data_full.iso3.isin(['RUS', 'RU-MOW'])].sort_values(['date', 'iso3']).tail(6)

,date,iso3,country,confirmed
14945,2020-05-01,RU-MOW,Moscow,57300
20399,2020-05-01,RUS,Russia,114431
14946,2020-05-02,RU-MOW,Moscow,62658
20400,2020-05-02,RUS,Russia,124054
14947,2020-05-03,RU-MOW,Moscow,68606
20401,2020-05-03,RUS,Russia,134687


## Сохраняем датасет

In [25]:
data_full.to_csv(os.path.join('data', f'full_data_{CURRENT_DATE}.csv'), index=False, encoding='utf-8')